## Imports és logger

In [23]:
# Importok
import os
import sys
import subprocess
import logging
import time
from pathlib import Path

In [24]:
def get_logger(name: str, filename: str):
    """Logger that writes to stdout and to OUTPUT_DIR/<filename>."""
    logger = logging.getLogger(name)
    if logger.handlers:
        return logger

    logger.setLevel(logging.INFO)
    fmt = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    stream = logging.StreamHandler(sys.stdout)
    stream.setFormatter(fmt)
    logger.addHandler(stream)

    try:
        output_dir = Path(os.getenv('OUTPUT_DIR', '../output'))
        output_dir.mkdir(parents=True, exist_ok=True)
        file_handler = logging.FileHandler(output_dir / filename)
        file_handler.setFormatter(fmt)
        logger.addHandler(file_handler)
    except Exception:
        # Ha a file handler nem elérhető, marad a stdout
        pass

    logger.propagate = False
    return logger

logger_api = get_logger('api_service', 'api.log')
logger_frontend = get_logger('frontend_service', 'frontend.log')

## Modellek ellenőrzése (API-hoz)

In [25]:
def check_models_exist():
    """Check if required models are available."""
    output_dir = Path(os.getenv('OUTPUT_DIR', '../output'))
    models_dir = output_dir / 'models'
    
    if not models_dir.exists():
        logger_api.error(f"Models directory not found: {models_dir}")
        return False
    
    pt_candidates = list(models_dir.glob('best_*.pt'))
    generic_best = models_dir / 'best_model.pt'
    label_map = models_dir / 'label_mapping.json'

    has_checkpoint = generic_best.exists() or len(pt_candidates) > 0
    if not has_checkpoint:
        logger_api.error("No model checkpoints found (expected best_model.pt or best_*.pt)")
        return False

    if not label_map.exists():
        logger_api.warning("label_mapping.json not found; will rely on checkpoint label2id if present")

    logger_api.info("Models found, ready to start API service")
    return True

## API elérhetőség ellenőrzése (Frontend-hez)

In [26]:
def check_api_available():
    """Check if API service is available."""
    try:
        import requests
        api_url = os.getenv('API_URL', 'http://localhost:8000')
        response = requests.get(f"{api_url}/", timeout=5)
        if response.status_code == 200:
            logger_frontend.info(f"API service is available at {api_url}")
            return True
    except Exception as e:
        logger_frontend.warning(f"API service not available: {e}")
    return False

## Környezeti változók (példa teszteléshez)

In [27]:
# Példa beállítások (szükség esetén módosítsd)
# os.environ['START_API_SERVICE'] = '1'
# os.environ['START_FRONTEND_SERVICE'] = '1'
os.environ['API_HOST'] = os.getenv('API_HOST', '0.0.0.0')
os.environ['API_PORT'] = os.getenv('API_PORT', '8000')
os.environ['API_URL'] = os.getenv('API_URL', 'http://localhost:8000')
os.environ['FRONTEND_HOST'] = os.getenv('FRONTEND_HOST', '0.0.0.0')
os.environ['FRONTEND_PORT'] = os.getenv('FRONTEND_PORT', '8501')

print('Env configured:')
print(' START_API_SERVICE:', os.getenv('START_API_SERVICE', 'Not set'))
print(' START_FRONTEND_SERVICE:', os.getenv('START_FRONTEND_SERVICE', 'Not set'))
print(' API_HOST:', os.getenv('API_HOST'))
print(' API_PORT:', os.getenv('API_PORT'))
print(' API_URL:', os.getenv('API_URL'))
print(' FRONTEND_HOST:', os.getenv('FRONTEND_HOST'))
print(' FRONTEND_PORT:', os.getenv('FRONTEND_PORT'))

Env configured:
 START_API_SERVICE: 1
 START_FRONTEND_SERVICE: 1
 API_HOST: 0.0.0.0
 API_PORT: 8000
 API_URL: http://localhost:8000
 FRONTEND_HOST: 0.0.0.0
 FRONTEND_PORT: 8501


## API service indítása (blokkoló)

In [28]:
def start_api_service():
    """Start the FastAPI service (blocking)."""
    logger_api.info('=' * 60)
    logger_api.info('Starting API Service (Step 07)')
    logger_api.info('=' * 60)
    
    start_service = os.getenv('START_API_SERVICE', '0').strip()
    if start_service not in ('1', 'true', 'True', 'TRUE', 'yes', 'Yes', 'YES'):
        logger_api.info('API service start is disabled (START_API_SERVICE not set to 1/true/yes)')
        logger_api.info('To enable, set environment variable: START_API_SERVICE=1')
        logger_api.info('Skipping API service startup.')
        return 0

    if not check_models_exist():
        logger_api.error('Cannot start API service without models!')
        return 1

    api_host = os.getenv('API_HOST', '0.0.0.0')
    api_port = int(os.getenv('API_PORT', '8000'))

    logger_api.info(f'Starting FastAPI server at {api_host}:{api_port}')
    logger_api.info('API will be accessible at:')
    logger_api.info(f'  - http://localhost:{api_port} (from host)')
    logger_api.info(f'  - http://{api_host}:{api_port} (from network)')
    logger_api.info('')
    logger_api.info('Press Ctrl+C to stop the server')
    logger_api.info('=' * 60)

    script_dir = Path('.').resolve() / 'src'
    api_app_path = script_dir / 'api' / 'app.py'
    if not api_app_path.exists():
        logger_api.error(f'API app not found at: {api_app_path}')
        return 1

    try:
        import uvicorn
        uvicorn.run("api.app:app", host=api_host, port=api_port, reload=False, log_level='info')
    except ImportError:
        logger_api.warning('uvicorn not found, trying subprocess method...')
        try:
            cmd = [sys.executable, '-m', 'uvicorn', 'api.app:app', '--host', api_host, '--port', str(api_port)]
            subprocess.run(cmd, check=True, cwd=script_dir)
        except subprocess.CalledProcessError as e:
            logger_api.error(f'Failed to start API service: {e}')
            return 1
        except FileNotFoundError:
            logger_api.error('uvicorn not installed! Install it with: pip install uvicorn')
            return 1
    except KeyboardInterrupt:
        logger_api.info('API service stopped by user')
        return 0
    except Exception as e:
        logger_api.error(f'Error starting API service: {e}')
        return 1

    return 0

print('API starter defined. Indításhoz: start_api_service() (blokkoló)')

API starter defined. Indításhoz: start_api_service() (blokkoló)


## Frontend service indítása (blokkoló)

In [29]:
def start_frontend_service():
    """Start the Streamlit frontend service (blocking)."""
    logger_frontend.info('=' * 60)
    logger_frontend.info('Starting Frontend Service (Step 08)')
    logger_frontend.info('=' * 60)

    start_service = os.getenv('START_FRONTEND_SERVICE', '0').strip()
    if start_service not in ('1', 'true', 'True', 'TRUE', 'yes', 'Yes', 'YES'):
        logger_frontend.info('Frontend service start is disabled (START_FRONTEND_SERVICE not set to 1/true/yes)')
        logger_frontend.info('To enable, set environment variable: START_FRONTEND_SERVICE=1')
        logger_frontend.info('Skipping frontend service startup.')
        return 0

    max_wait = 60
    waited = 0
    while not check_api_available() and waited < max_wait:
        if waited == 0:
            logger_frontend.warning('API service is not available! Waiting for it to become available...')
        time.sleep(5)
        waited += 5
    if waited >= max_wait:
        logger_frontend.warning(f'API service is still not available after {max_wait} seconds! Frontend will start, but may not work correctly.')
    else:
        logger_frontend.info(f'API became available after {waited} seconds.')

    frontend_host = os.getenv('FRONTEND_HOST', '0.0.0.0')
    frontend_port = int(os.getenv('FRONTEND_PORT', '8501'))
    api_url = os.getenv('API_URL', 'http://localhost:8000')

    logger_frontend.info(f'Starting Streamlit frontend at {frontend_host}:{frontend_port}')
    logger_frontend.info(f'Frontend will connect to API at: {api_url}')
    logger_frontend.info('Frontend will be accessible at:')
    logger_frontend.info(f'  - http://localhost:{frontend_port} (from host)')
    logger_frontend.info(f'  - http://{frontend_host}:{frontend_port} (from network)')
    logger_frontend.info('')
    logger_frontend.info('Press Ctrl+C to stop the server')
    logger_frontend.info('=' * 60)

    script_dir = Path('.').resolve() / 'src'
    frontend_app_path = script_dir / 'frontend' / 'app.py'
    if not frontend_app_path.exists():
        logger_frontend.error(f'Frontend app not found at: {frontend_app_path}')
        return 1

    try:
        cmd = [
            sys.executable, '-m', 'streamlit', 'run', str(frontend_app_path),
            f'--server.port={frontend_port}', f'--server.address={frontend_host}', '--server.headless=true'
        ]
        logger_frontend.info('Running command: ' + ' '.join(cmd))
        subprocess.run(cmd, check=True, cwd=script_dir)
    except subprocess.CalledProcessError as e:
        logger_frontend.error(f'Failed to start frontend service: {e}')
        return 1
    except FileNotFoundError:
        logger_frontend.error('streamlit not installed! Install it with: pip install streamlit')
        return 1
    except KeyboardInterrupt:
        logger_frontend.info('Frontend service stopped by user')
        return 0
    except Exception as e:
        logger_frontend.error(f'Error starting frontend service: {e}')
        return 1

    return 0

print('Frontend starter defined. Indításhoz: start_frontend_service() (blokkoló)')

Frontend starter defined. Indításhoz: start_frontend_service() (blokkoló)


## Gyors tesztek (API/Frontend)

In [30]:
def test_api_connection():
    """Test if API is available."""
    try:
        import requests
        api_url = os.getenv('API_URL', 'http://localhost:8000')
        print(f'Testing connection to {api_url}...')
        response = requests.get(f'{api_url}/', timeout=5)
        if response.status_code == 200:
            print(f'✓ API is available at {api_url}')
            print('Response:', response.json())
            return True
        else:
            print(f'✗ API returned status code: {response.status_code}')
            return False
    except ImportError:
        print('✗ requests not installed. pip install requests')
        return False
    except Exception as e:
        print(f'✗ API not available: {e}')
        return False

def test_frontend_connection():
    """Test if Frontend is available."""
    try:
        import requests
        frontend_url = os.getenv('FRONTEND_URL', 'http://localhost:8501')
        print(f'Testing connection to {frontend_url}...')
        response = requests.get(frontend_url, timeout=5)
        if response.status_code == 200:
            print(f'✓ Frontend is available at {frontend_url}')
            return True
        else:
            print(f'✗ Frontend returned status code: {response.status_code}')
            return False
    except ImportError:
        print('✗ requests not installed. pip install requests')
        return False
    except Exception as e:
        print(f'✗ Frontend not available: {e}')
        return False

# Tesztek futtatása
print('='*60)
print('Testing connections...')
print('='*60)
test_api_connection()
print()
test_frontend_connection()

Testing connections...
Testing connection to http://localhost:8000...
✗ API not available: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x738380428c10>: Failed to establish a new connection: [Errno 111] Connection refused'))

Testing connection to http://localhost:8501...
✗ Frontend not available: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7383901ab850>: Failed to establish a new connection: [Errno 111] Connection refused'))


False